# Eff. matching to GEN particle


   * [Efficiency vs $\eta$](#Effinciency-vs-$\eta$)
   * [Efficiency vs $p_{T}$](#Efficiency-vs-$p_{T}$)
   * [Turn-on curves](#Turn-on-curves)


In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load ../drawingTools
import ROOT
import math
import uuid

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 6)
stuff = []
f_idx = 0

ROOT.gStyle.SetOptTitle(False)
ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.13)

# ROOT.gStyle.SetCanvasBorderMode(0)
# ROOT.gStyle.SetCanvasColor(0)


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext

def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend

def newCanvas(name=None, title=None, height=600, width=800, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title, width, height
    canvas = ROOT.TCanvas(name, title, width, height)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas

def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


def draw(histograms,
         labels,
         options='',
         norm=False,
         logy=False,
         min_y=None,
         max_y=None,
         text=None,
         y_axis_label=None,
         x_axis_label=None,
         v_lines=None,
         h_lines=None,
         do_stats=False,
         do_profile=False,
         do_ratio=False):
    global colors
    global stuff
    global p_idx


    # 0 - determine kind and # of histos
    n_histos = len(histograms)
    if n_histos == 0:
        print '[draw]**ERROR: empy histo list'
        return

    histo_class = histograms[0].ClassName()

    # 1 - check argument consisntency
    if 'TH2' not in histo_class and do_profile:
        print '[draw]**ERROR: do_profile set for histo of class: {}'.format(histo_class)
        return

    # TH1 histograms are overlayed
    # TH2 histograms are drawn side by side in the same canvas
    overlay = True
    if n_histos > 1:
        if 'TH2' in histo_class or 'TH3' in histo_class:
            overlay = False

    # 2 - determine canvas properties
    xdiv = 0
    ydiv = 0
    c_width = 800
    c_height = 600
    # do_ratio = False

    if do_ratio:
        c_height = 800

    if not overlay:
        xdiv = 2
        c_width = 1000.
        ydiv = math.ceil(float(n_histos)/2)
        c_height = 500*ydiv
    canvas = newCanvas(name=None,
                       title=None,
                       height=int(c_height),
                       width=int(c_width),
                       xdiv=int(xdiv),
                       ydiv=int(ydiv))
    if overlay:
        canvas.SetRightMargin(0.30)

    canvas.cd()
    leg = getLegend()

    drawn_histos = []
    # drawn_histos = histograms
    for hidx, hist in enumerate(histograms):
        if 'TGraph' not in histo_class:
            hist.SetStats(do_stats)
        opt = options
        if overlay:
            hist.SetLineColor(colors[hidx])
            if hidx:
                opt = 'same,'+options
            else:
                if 'TGraph' in histo_class:
                    opt = options+'PA'
        else:
            canvas.cd(hidx+1)
        # print hidx, opt
        d_hist = hist
        if norm:
            d_hist = hist.DrawNormalized(opt, 1.)
        else:
            hist.Draw(opt)

        if do_profile:
            profname = d_hist.GetName()+'_prof_'+str(p_idx)
            p_idx += 1
            firstbin = 1
            lastbin = -1
            prof = d_hist.ProfileX(profname, firstbin, lastbin, 's')
            prof.SetMarkerColor(2)
            prof.SetLineColor(2)
            prof.Draw('same')

        if overlay:
            leg.AddEntry(hist, labels[hidx], 'l')
        else:
            if text:
                newtext = '{}: {}'.format(labels[hidx], text)
                rtext = getText(newtext, 0.15, 0.85)
                rtext.Draw('same')
        drawn_histos.append(d_hist)

    if do_ratio:
        h_ratio = ROOT.TRatioPlot(drawn_histos[0], drawn_histos[1])
        stuff.append(h_ratio)
        h_ratio.Draw()

        pad = canvas.cd(1)
        h_ratio.GetUpperPad().SetBottomMargin(0)
        h_ratio.GetUpperPad().SetRightMargin(0.3)
        if logy:
            h_ratio.GetUpperPad().SetLogy()
        pad.Update()
        pad = canvas.cd(2)
        h_ratio.GetLowerPad().SetTopMargin(0)
        h_ratio.GetLowerPad().SetRightMargin(0.3)
        pad.Update()
        # hratio.GetLowerRefGraph().GetYaxis().SetRangeUser(-6, 6)

    canvas.Update()

    # we now set the axis properties
    max_value = max_y
    min_value = min_y

    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in drawn_histos])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in drawn_histos])*1.2
    # print min_value, max_value

    for hist in drawn_histos:
        hist.GetYaxis().SetRangeUser(min_value, max_value)
        if y_axis_label:
            hist.GetYaxis().SetTitle(y_axis_label)
        if x_axis_label:
            hist.GetXaxis().SetTitle(x_axis_label)

    canvas.Draw()

    # we draw additional stuff if needed
    if overlay:
        canvas.cd()
        leg.Draw()
        if text:
            rtext = getText(text, 0.15, 0.85)
            rtext.Draw("same")

    pad_range = range(0, 1)
    if not overlay:
        pad_range = range(1, len(histograms)+1)

    for pad_id in pad_range:
        canvas.cd(pad_id)
        if v_lines:
            for v_line_x in v_lines:
                aline = ROOT.TLine(v_line_x, ROOT.gPad.GetUymin(), v_line_x,  ROOT.gPad.GetUymax())
                aline.SetLineStyle(2)
                aline.Draw("same")
                stuff.append(aline)
        if h_lines:
            for h_line_y in h_lines:
                aline = ROOT.TLine(ROOT.gPad.GetUxmin(), h_line_y, ROOT.gPad.GetUxmax(),  h_line_y)
                aline.SetLineStyle(2)
                aline.Draw("same")
                stuff.append(aline)
        if logy:
            if not do_ratio:
                ROOT.gPad.SetLogy()

        ROOT.gPad.Update()
    canvas.Update()
    canvas.Draw()

    return canvas


In [3]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v76t'

files = {}
file_keys = {}


class RootFile:
    def __init__(self, file_name):
        global file
        self.file_name = file_name
        if self.file_name not in files.keys():
            print 'get file: {}'.format(self.file_name)
            files[self.file_name] = ROOT.TFile(self.file_name)
        self._file = files[self.file_name]
        self._file_keys = None

    def cd(self):
        self._file.cd()

    def GetListOfKeys(self):
        global file_keys
        if self.file_name not in file_keys.keys():
            print 'get list'
            file_keys[self.file_name] = self._file.GetListOfKeys()
        self._file_keys = file_keys[self.file_name]
        return self._file_keys


class Sample():
    def __init__(self, name, label, version=None, type=None):
        self.name = name
        self.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        self.histo_filename = '../plots1/histos_{}{}.root'.format(self.name, version)
        self.histo_file = ROOT.TFile(self.histo_filename, 'r')
        self.type = type


# sample_names = ['ele_flat2to100_PU0',
#                 'ele_flat2to100_PU200',
#                 'photonPt35_PU0',
#                 'photonPt35_PU200']


def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary


class HProxy:
    def __init__(self, classtype, tp, tp_sel, gen_sel, root_file):
        self.classtype = classtype
        self.tp = tp
        self.tp_sel = tp_sel
        self.gen_sel = gen_sel
        self.root_file = root_file
        self.instance = None

    def get(self):
        if self.instance is None:
            name = '{}_{}_{}'.format(self.tp, self.tp_sel, self.gen_sel)
            if self.gen_sel is None:
                name = '{}_{}'.format(self.tp, self.tp_sel)
            self.instance = self.classtype(name, self.root_file)
        return self.instance


class HPlot:
    def __init__(self, samples, tp_sets, tp_selections, gen_selections):
        self.tp_sets = tp_sets
        self.tp_selections = tp_selections
        self.gen_selections = gen_selections
        self.pus = []
        self.labels_dict = {}

        for sample in samples:
            self.pus.append(sample.label)
            self.labels_dict[sample.type] = sample.type

        self.data = pd.DataFrame(columns=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype', 'histo'])

        self.labels_dict.update(tp_sets)
        self.labels_dict.update(tp_selections)
        self.labels_dict.update(gen_selections)
        self.labels_dict.update({'PU0': 'PU0', 'PU200': 'PU200'})

    def cache_histo(self,
                    classtype,
                    samples,
                    pus,
                    tps,
                    tp_sels,
                    gen_sels):
        if gen_sels is None:
            gen_sels = [None]

        for sample in samples:
            print sample
            for tp in tps:
                for tp_sel in tp_sels:
                    for gen_sel in gen_sels:
                        print sample, tp, tp_sel, gen_sel
                        self.data = self.data.append({'sample': sample.type,
                                                      'pu': sample.label,
                                                      'tp': tp,
                                                      'tp_sel': tp_sel,
                                                      'gen_sel': gen_sel,
                                                      'classtype': classtype,
                                                      'histo': HProxy(classtype, tp, tp_sel, gen_sel, sample.histo_file)},
                                                     ignore_index=True)

    def get_histo(self,
                  classtype,
                  sample=None,
                  pu=None,
                  tp=None,
                  tp_sel=None,
                  gen_sel=None):
        histo = None
        labels = []
        text = ''

        query = '(pu == @pu) & (tp == @tp) & (tp_sel == @tp_sel) & (classtype == @classtype)'
        if gen_sel is not None:
            query += ' & (gen_sel == @gen_sel)'
        else:
            query += ' & (gen_sel.isnull())'
        if sample is not None:
            query += '& (sample == @sample)'

        histo_df = self.data.query(query)

        if histo_df.empty:
            print 'No match found for: pu: {}, tp: {}, tp_sel: {}, gen_sel: {}, classtype: {}'.format(pu, tp, tp_sel, gen_sel, classtype)
            return None, None, None
#         print histo_df

        field_counts = histo_df.apply(lambda x: len(x.unique()))
        label_fields = []
        text_fields = []
        # print field_counts
        for field in field_counts.iteritems():
            if(field[1] > 1 and field[0] != 'histo'):
                label_fields.append(field[0])
            if(field[1] == 1 and field[0] != 'histo' and field[0] != 'classtype' and field[0] != 'sample'):
                if(gen_sel is None and field[0] == 'gen_sel'):
                    continue
                text_fields.append(field[0])
#         print 'label fields: {}'.format(label_fields)
#         print 'text fields: {}'.format(text_fields)

        for item in histo_df[label_fields].iterrows():
            labels.append(', '.join([self.labels_dict[tx] for tx in item[1].values if self.labels_dict[tx] != '']))

        # print labels
        text = ', '.join([self.labels_dict[fl] for fl in histo_df[text_fields].iloc[0].values if self.labels_dict[fl] != ''])
        histo = [his.get() for his in histo_df['histo'].values]
        return histo, labels, text


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'ele')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version, 'photon')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

tpsets = {'DEF': 'NNDR',
          'DEFCalib': 'NNDR Calib v1'}

tpset_selections = {}
gen_selections = {}
samples = []

# tpset_selections.update(get_label_dict(tp_rate_selections))
tpset_selections.update(get_label_dict(selections.tp_match_selections))
gen_selections.update(get_label_dict(selections.gen_part_selections))
gen_selections.update({'nomatch': ''})


gen_part_selections: 10


Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU0_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU200_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_pion_flat2to100_PU0_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu0_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v76t.root does not exist


In [4]:
tpsets = {'L1Trk': 'L1Track'}

tpset_selections = {}

gen_selections = {}
samples = []


#tpset_selections.update(get_label_dict(tp_rate_selections))
tpset_selections.update(get_label_dict(selections.tracks_selections))

gen_selections.update(get_label_dict(selections.gen_part_selections))

samples = samples_ele

In [5]:
tpset_selections

{'Pt10': 'p_{T}^{tk} > 10 GeV', 'Pt2': 'p_{T}^{tk} > 2 GeV', 'all': ''}

In [6]:
%%time
# for key in samples_ele[0].histo_file.GetListOfKeys():
#     #pass
#     key.GetName()
# print len(samples_ele[0].histo_file.GetListOfKeys())
# l = [histo for histo in samples_ele[0].histo_file.GetListOfKeys()]
l = [(histo.ReadObj(), histo.GetName()) for histo in samples_ele[0].histo_file.GetListOfKeys() if 'DEF_all'+'_' in histo.GetName()]

CPU times: user 11.3 ms, sys: 484 µs, total: 11.8 ms
Wall time: 12.2 ms


In [7]:
%%time

hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)
samples = samples_ele


hplot.cache_histo(classtype=histos.HistoSetEff, 
                  samples=samples,
                  pus=[],
                  tps=tpsets,
                  tp_sels=tpset_selections,
                  gen_sels=gen_selections)



<__main__.Sample instance at 0x11698bfc8>
<__main__.Sample instance at 0x11698bfc8> L1Trk all GENPt40
<__main__.Sample instance at 0x11698bfc8> L1Trk all GENEtaC
<__main__.Sample instance at 0x11698bfc8> L1Trk all GENEtaB
<__main__.Sample instance at 0x11698bfc8> L1Trk all GENEtaD
<__main__.Sample instance at 0x11698bfc8> L1Trk all GENEtaBC
<__main__.Sample instance at 0x11698bfc8> L1Trk all GENPt30
<__main__.Sample instance at 0x11698bfc8> L1Trk all GENPt20
<__main__.Sample instance at 0x11698bfc8> L1Trk all GENEtaBCD
<__main__.Sample instance at 0x11698bfc8> L1Trk all GENPt10
<__main__.Sample instance at 0x11698bfc8> L1Trk all GEN
<__main__.Sample instance at 0x11698bfc8> L1Trk Pt10 GENPt40
<__main__.Sample instance at 0x11698bfc8> L1Trk Pt10 GENEtaC
<__main__.Sample instance at 0x11698bfc8> L1Trk Pt10 GENEtaB
<__main__.Sample instance at 0x11698bfc8> L1Trk Pt10 GENEtaD
<__main__.Sample instance at 0x11698bfc8> L1Trk Pt10 GENEtaBC
<__main__.Sample instance at 0x11698bfc8> L1Trk Pt10 

In [9]:
%%time

for pu in ['PU0', 'PU200']:
    for tp in tpsets:
        for tp_sel in tpset_selections:
            tp_sel_den = 'all'
            if 'Pt' in tp_sel:
                # print tp_sel
                tp_sel_den = tp_sel.split('Pt')[0]
                if tp_sel_den == '':
                    tp_sel_den = 'all'
                print '{} den -> {}'.format(tp_sel, tp_sel_den) 
            else:
                continue
            for gen_sel in ['GEN', 'GENEtaBC']:
                hsetden = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tp, tp_sel_den, gen_sel)
                hset = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tp, tp_sel, gen_sel)
                hset[0][0].computeTurnOn(hsetden[0][0].h_num)


Pt10 den -> all
Pt2 den -> all
Pt10 den -> all
Pt2 den -> all
CPU times: user 2.3 s, sys: 302 ms, total: 2.6 s
Wall time: 2.72 s


In [10]:
hplot.data

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,ele,PU0,L1Trk,all,GENPt40,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x115ec98c0>
1,ele,PU0,L1Trk,all,GENEtaC,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x115ec9ef0>
2,ele,PU0,L1Trk,all,GENEtaB,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x115ec9dd0>
3,ele,PU0,L1Trk,all,GENEtaD,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x116928d40>
4,ele,PU0,L1Trk,all,GENEtaBC,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x116928998>
5,ele,PU0,L1Trk,all,GENPt30,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x105763878>
6,ele,PU0,L1Trk,all,GENPt20,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x116928320>
7,ele,PU0,L1Trk,all,GENEtaBCD,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11699eb90>
8,ele,PU0,L1Trk,all,GENPt10,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x116956950>
9,ele,PU0,L1Trk,all,GEN,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11699e290>


In [11]:
tps = ['L1Trk']
tp_select = {}
tp_select['L1Trk'] = [ 'all', 'Pt2', 'Pt10']
tp_select['TkEG'] = [ 'EGq2M2', 'EGq3M2',  'EGq2M3', 'EGq3M3',  'EGq2M4', 'EGq3M4']

# gen_select = ['GEN', 'GENEtaA', 'GENEtaB', 'GENEtaC', 'GENEtaD', 'GENEtaE',
#               'GENEtaAB', 'GENEtaABC', 'GENEtaBC', 'GENEtaBCD', 'GENEtaBCDE' ]
gen_select = ['GENEtaBC']


## Effinciency vs $\eta$

In [32]:
line_m1p52 = ROOT.TLine(-1.52,0, -1.52, 1.1)
line_m1p52.SetLineStyle(2)
line_p1p52 = ROOT.TLine(1.52,0, 1.52, 1.1)
line_p1p52.SetLineStyle(2)
line_m1p7 = ROOT.TLine(-1.7,0, -1.7, 1.1)
line_m1p7.SetLineStyle(2)
line_p1p7 = ROOT.TLine(1.7,0, 1.7, 1.1)
line_p1p7.SetLineStyle(2)
line_m2p8 = ROOT.TLine(-2.8,0, -2.8, 1.1)
line_m2p8.SetLineStyle(2)
line_p2p8 = ROOT.TLine(2.8,0, 2.8, 1.1)
line_p2p8.SetLineStyle(2)

In [12]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in ['GEN']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
#             print [his.h_num.h_eta.GetName() for his in hsets]

In [13]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
     

In [14]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in ['GENPt20']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [15]:
for tp in tps:
    for pu in ['PU0', 'PU200']:
#     for tp_sel in tp_select[tp]:
        for gen_sel in ['GENPt20']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tp, ['Pt2', 'Pt10'], gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


## Efficiency vs $p_{T}$

In [37]:
line_y1p0 = ROOT.TLine(0.,1., 110., 1.)
line_y0p95 = ROOT.TLine(0.,0.95, 110., 0.95)
line_y0p95.SetLineStyle(2)

In [16]:
for tp in tps:
    # for pu in ['PU0', 'PU200']:
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])
   

In [17]:
for tp in tps:
    for pu in ['PU0', 'PU200']:
#     for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tp, ['Pt2', 'Pt10'], gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])
   

## Turn-on curves

In [40]:
for tp in tps:
    for tp_sel in ['Pt20', 'EmPt20', 'Emv1Pt20', 'Pt30', 'EmPt30', 'Emv1Pt30']:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            drawSame([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

No match found for: pu: ['PU0', 'PU200'], tp: L1Trk, tp_sel: Pt20, gen_sel: GENEtaBC, classtype: python.l1THistos.HistoSetEff


NameError: name 'drawSame' is not defined

In [ ]:
for pu in ['PU0', 'PU200']:
    for tp in tps:
        for tp_sel in ['Pt20', 'EmPt20', 'Emv1Pt20', 'Pt30', 'EmPt30', 'Emv1Pt30']:
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tp, tp_sel, ['GENEtaABC', 'GENEtaBC'])
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                drawSame([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

In [ ]:
for pu in ['PU0', 'PU200']:
        for tp_sel in ['EmPt20', 'Emv1Pt20', 'EmPt30', 'Emv1Pt30']:
            for gen_sel in gen_select:
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, ['DEF', 'DEFCalib'], tp_sel, gen_sel)
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                drawSame([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

In [ ]:
for pu in ['PU0', 'PU200']:
        for tp_sel in ['EmPt20', 'Emv1Pt20', 'EmPt30', 'Emv1Pt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, ['DEF', 'DEFCalib'], tp_sel, ['GENEtaABC', 'GENEtaBC'])
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            drawSame([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

## Scaling

In [ ]:
def f_yc(x, par):
    return (ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], 0) - ROOT.TMath.Exp(-par[0]*(x[0]-par[1])+par[0]*par[0]*par[2]*par[2]/2)*ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], par[0]*par[0]*par[2]*par[2])) * (par[3] - par[4]) + par[4]

tf_yc = ROOT.TF1('tf_yc', f_yc, 0, 100, 5)
tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
tf_yc.SetParNames("#lambda", "#mu", "#sigma", "Plateau", "Baseline");
tf_yc.SetLineColor(6)
tf_yc.FixParameter(4,0)

tf_yc.SetParLimits(3, 0.8, 1)
tf_yc.SetParLimits(0, 0, 1)

c = newCanvas()
tf_yc.Draw()
c.Draw()


In [ ]:

import numpy as np
from array import array

def f_cb(x, par):
    return ROOT.Math.crystalball_function(x[0], par[0], par[1], par[2], par[3])

tf_cb = ROOT.TF1('cb', f_cb, 0, 100, 4)
tf_cb.SetParameters(10., 4., 3, 20.)

def f_cbi(x, par):
    return ROOT.Math.crystalball_cdf(x[0], par[0], par[1], par[2], par[3])
#     tf_cb.SetParameters(par[0], par[1], par[2], par[3])
#     npts = 1000
#     x_ = []
#     w_ = []
   
#     return tf_cb.IntegralFast(npts, np.array(npts), np.array(npts), 0., x[0])
    #return tf_cb.Integral(0., x[0])
    
    
tf_cbi = ROOT.TF1('cbi', f_cbi, 0, 100, 4)
tf_cbi.SetParameters(10., 4., 3, 20.)
tf_cbi.SetLineColor(4)

c = newCanvas()
tf_cbi.Draw()
c.Draw()


    # ROOT::Math::crystalball_function (double x, double alpha, double n, double sigma, double x0=0)

# fitf_cb = ROOT.TF1('cbfunc', 'ROOT.TMath.crystalball_function (x, [0], [1], [2], [3])', 0, 100)
# 

# fitf_cb.SetParameters(1, 1, 1, 0)


In [ ]:
# Double_t turnon_func(Double_t *x, Double_t *par)
# {
#   double halfpoint = par[0];
#   double slope = par[1];
#   double plateau = par[2];

#   //double offset = par[3];
#   //double plateau = 1.0;
#   double offset = 0;

#   double pt = TMath::Max(x[0],0.000001);

#    double arg = 0;
#    //cout << pt <<", "<< halfpoint <<", " << slope <<endl;
#    arg = (pt - halfpoint)/(TMath::Sqrt(pt)*slope);
#    double fitval = offset+0.5*plateau*(1+TMath::Erf(arg));
#    return fitval;
# }




def turnon_func1(x, par):

    return 0.5 * par[0] * (1+ROOT.TMath.Erf((x[0]-par[1])/(par[2]*ROOT.TMath.Sqrt(2))))




def turnon_func(x, par):
  
    halfpoint = par[0]
    slope = par[1]
    plateau = par[2]
    offset = 0
    pt = ROOT.TMath.Max(x[0],0.000001)
    arg = 0
    arg = (pt - halfpoint)/(ROOT.TMath.Sqrt(pt)*slope)
    fitval = offset+0.5*plateau*(1+ROOT.TMath.Erf(arg))
    return fitval

fitf_to = ROOT.TF1('myfunc', turnon_func1, 0, 100, 3)
#fitf.SetParNames('#epsilon', "#mu", '#sigma')
fitf_to.SetParameters(1,25,2)
fitf_to.SetLineColor(3)

def fermi(x, params):
    return params[3]*(params[0] - 1/(ROOT.TMath.Exp((x[0]-params[1])/(params[2]))+1))
  
fitf = ROOT.TF1('myfunc', fermi, 0, 100, 4)
fitf.SetParNames('#epsilon', "#mu", '#sigma')
fitf.SetParameters(1,25,2,1)



c = newCanvas()
fitf.Draw()
c.Draw()


In [ ]:
ROOT.TVirtualFitter.SetDefaultFitter("Fumili2")
ROOT.TVirtualFitter.SetPrecision(1e-04)

points = []


for pu in ['PU0', 'PU200']:
    for tp in ['DEF', 'DEFCalib']:
        for pt in [10, 20, 25, 30]:
                tp_sel = 'EmPt{}'.format(pt)
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tp, tp_sel, 'GENEtaBC')
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                h_eff_vs_pt = [his.h_eff.h_pt.CreateGraph() for his in hsets]
                tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
                result = h_eff_vs_pt[0].Fit(tf_yc, 'MQRS+')
                print '------- {}'.format( text)
                print result.Print()
                pt095 = tf_yc.GetX(0.95)
                points.append((tp, pu, pt, pt095))
                drawSame(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                ROOT.gStyle.SetOptFit(11111)

In [ ]:
# print points

points_calib_pu0 = [point for point in points if ('Calib' in point[0] and 'PU0' in point[1])]
points_calib_pu200 = [point for point in points if ( 'Calib' in point[0] and 'PU200' in point[1])]

points_nocalib_pu0 = [point for point in points if ('Calib' not in point[0] and 'PU0' in point[1])]
points_nocalib_pu200 = [point for point in points if ( 'Calib' not in point[0] and 'PU200' in point[1])]

# print points
# print points_calib_pu0
# print points_calib_pu200
# print points_nocalib_pu0
# print points_nocalib_pu200

graph_nocalib_pu0 = ROOT.TGraph(len(points_nocalib_pu0), array('d', [point[2] for point in points_nocalib_pu0]), array('d',[point[3] for point in points_nocalib_pu0]))
drawGraphsSame([graph_nocalib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='no calib')
graph_nocalib_pu0.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

graph_calib_pu0 = ROOT.TGraph(len(points_calib_pu0), array('d', [point[2] for point in points_calib_pu0]), array('d',[point[3] for point in points_calib_pu0]))
drawGraphsSame([graph_calib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='calib')
graph_calib_pu0.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)


graph_nocalib_pu200 = ROOT.TGraph(len(points_nocalib_pu200), array('d', [point[2] for point in points_nocalib_pu200]), array('d',[point[3] for point in points_nocalib_pu200]))
drawGraphsSame([graph_nocalib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='no calib')
graph_nocalib_pu200.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

graph_calib_pu200 = ROOT.TGraph(len(points_calib_pu200), array('d', [point[2] for point in points_calib_pu200]), array('d',[point[3] for point in points_calib_pu200]))
drawGraphsSame([graph_calib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='calib')
graph_calib_pu200.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

In [ ]:
points = []


for pu in ['PU0', 'PU200']:
    for tp in ['DEF', 'DEFCalib']:
        for pt in [10, 20, 25, 30]:
                tp_sel = 'EmPt{}'.format(pt)
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tp, tp_sel, 'GENEtaABC')
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                h_eff_vs_pt = [his.h_eff.h_pt.CreateGraph() for his in hsets]
                tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
                result = h_eff_vs_pt[0].Fit(tf_yc, 'MQRS+')
                print '------- {}'.format( text)
                print result.Print()
                pt095 = tf_yc.GetX(0.95)
                points.append((tp, pu, pt, pt095))
                drawSame(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                ROOT.gStyle.SetOptFit(11111)

In [ ]:
points_calib_pu0 = [point for point in points if ('Calib' in point[0] and 'PU0' in point[1])]
points_calib_pu200 = [point for point in points if ( 'Calib' in point[0] and 'PU200' in point[1])]

points_nocalib_pu0 = [point for point in points if ('Calib' not in point[0] and 'PU0' in point[1])]
points_nocalib_pu200 = [point for point in points if ( 'Calib' not in point[0] and 'PU200' in point[1])]

print points
# print points_calib_pu0
# print points_calib_pu200
# print points_nocalib_pu0
# print points_nocalib_pu200

graph_nocalib_pu0 = ROOT.TGraph(len(points_nocalib_pu0), array('d', [point[2] for point in points_nocalib_pu0]), array('d',[point[3] for point in points_nocalib_pu0]))
drawGraphsSame([graph_nocalib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='no calib')
graph_nocalib_pu0.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

graph_calib_pu0 = ROOT.TGraph(len(points_calib_pu0), array('d', [point[2] for point in points_calib_pu0]), array('d',[point[3] for point in points_calib_pu0]))
drawGraphsSame([graph_calib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='calib')
graph_calib_pu0.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)


graph_nocalib_pu200 = ROOT.TGraph(len(points_nocalib_pu200), array('d', [point[2] for point in points_nocalib_pu200]), array('d',[point[3] for point in points_nocalib_pu200]))
drawGraphsSame([graph_nocalib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='no calib')
graph_nocalib_pu200.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

graph_calib_pu200 = ROOT.TGraph(len(points_calib_pu200), array('d', [point[2] for point in points_calib_pu200]), array('d',[point[3] for point in points_calib_pu200]))
drawGraphsSame([graph_calib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='calib')
graph_calib_pu200.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

In [ ]:

samples = samples_ele
tp_sets = ['DEF', 'DEF_em', 'DEF_pt20', 'DEF_pt20_em', 'DEF_pt25', 'DEF_pt25_em', 'DEF_pt30', 'DEF_pt30_em']
particles = ['ele', 'eleA', 'eleB', 'eleC', 'eleD', 'eleAAA', 'eleAA', 'eleBB', 'eleCC', 'eleDD']


labels = [sample.label for sample in samples]




histo_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_num_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_den_df = pd.DataFrame(index=tp_sets, columns=particles)
titles_df = pd.DataFrame(index=tp_sets, columns=particles)
# print histo_num_df
for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]
        histo_num_df.loc[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num for sample in samples]
        histo_den_df.loc[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]

        titles_df.loc[tp_set, particle] = '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])

histo_df



In [ ]:
titles_df

In [ ]:
dir(histo_df.ele['DEF'][0])


In [ ]:
histo_df.loc['DEF', 'ele']

In [ ]:

for particle in ['ele']:
    for tp_set in tp_sets:
        h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.ele[tp_set]]
        drawGraphsSame(h_eff_vs_eta, labels, text=titles_df.loc[tp_set, particle])



In [ ]:
line095 = ROOT.TLine(0,0.95,110,0.95)
line095.SetLineStyle(2)
for particle in particles:
    for tp_set in tp_sets:
        #print 'particle: {}, TP set: {}, text: {}'.format(particle, tp_set, titles_df.loc[tp_set, particle])
        h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histo_df.loc[tp_set, particle]]
        drawGraphsSame(h_eff_vs_pt, labels, text=titles_df.loc[tp_set, particle], min_y=0., max_y=1.1)
        line095.Draw('same')

In [ ]:
histo_den_df

In [ ]:
# build turnon

for particle in particles:
    TP_set_den = 'DEF' # -> L1 matched/GEN
    TP_set_num_l = ['DEF_pt20', 'DEF_pt25', 'DEF_pt30'] # -> L1 matched & above threshols / GEN
    
    histo_den = [histo for histo in histo_num_df.loc[TP_set_den][particle]] # -> L1 matched
    for TP_set_num in TP_set_num_l:
        histo_num = [histo for histo in histo_num_df.loc[TP_set_num][particle]] # -> L1 matched & above threshols

        histos_turnon = []
        for idx,histo_num in enumerate(histo_num):
            histo_turnon_num = histo_num.h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_num)
            histo_turnon_den = histo_den[idx].h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_den)
            hton = ROOT.TEfficiency(histo_turnon_num, histo_turnon_den)
            stuff.append(hton)
            histos_turnon.append(hton)
        drawGraphsSame(histos_turnon, labels, text=titles_df.loc[TP_set_num, particle])

In [ ]:
for particle in particles:
    TP_set_den = 'DEF_em' # -> L1 matched/GEN
    TP_set_num_l = ['DEF_pt20_em', 'DEF_pt25_em', 'DEF_pt30_em'] # -> L1 matched & above threshols / GEN
    
    histo_den = [histo for histo in histo_num_df.loc[TP_set_den][particle]] # -> L1 matched
    for TP_set_num in TP_set_num_l:
        histo_num = [histo for histo in histo_num_df.loc[TP_set_num][particle]] # -> L1 matched & above threshols

        histos_turnon = []
        for idx,histo_num in enumerate(histo_num):
            histo_turnon_num = histo_num.h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_num)
            histo_turnon_den = histo_den[idx].h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_den)
            hton = ROOT.TEfficiency(histo_turnon_num, histo_turnon_den)
            stuff.append(hton)
            histos_turnon.append(hton)
        drawGraphsSame(histos_turnon, labels, text=titles_df.loc[TP_set_num, particle])

# Photon pt35 GeV 1.6 < |eta| < 2.8 

In [ ]:

samples = samples_photon
tp_sets = ['DEF', 'DEF_em']
particles = ['photon']
labels = [sample.label for sample in samples]

histo_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_num_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_den_df = pd.DataFrame(index=tp_sets, columns=particles)

for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]
        histo_num_df[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num for sample in samples]
        histo_den_df[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]

        histo_df



In [ ]:
dir(histo_df.photon['DEF'][0])


In [ ]:
tp_set = 'DEF'
h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.photon[tp_set]]
drawGraphsSame(h_eff_vs_eta, labels)

tp_set = 'DEF_em'
h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.photon[tp_set]]
drawGraphsSame(h_eff_vs_eta, labels)
